In [ ]:
from PIL import Image

dir_raw = "man_shirt"
dir = f"{dir_raw}"
image = Image.open(f"{dir}/source_foreground.jpg")
masked_image = Image.open(f"{dir}/destination_raw_image.jpg")
w = masked_image.width
h = masked_image.height
masked_image = masked_image
mask = Image.open(f"{dir}/destination_foreground_mask.jpg").resize((w, h))

additional_prompt_positive = "t-shirt, male body"
additional_prompt_negative = ""

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

image_grid([image, masked_image, mask], 1, 3)

In [ ]:
from diffusers import StableDiffusionInpaintPipeline
from diffusers import AutoencoderKL
from ipadapter.ip_adapter import IPAdapter

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema"),
    feature_extractor=None,
    safety_checker=None
).to("cpu")

img_encoder_path = "ipadapter/models/image_encoder/"
ip_cpkt = "ipadapter/models/ip-adapter_sd15_light.bin"
ip_model = IPAdapter(pipe, img_encoder_path, ip_cpkt, "cpu")


In [ ]:
import os
import shutil
from matplotlib import pyplot as plt


images = ip_model.generate(
    pil_image=image,
    num_samples=1,
    num_inference_steps=50,
    seed=42,
    image=masked_image,
    mask_image=mask,
    strength=0.75,
    prompt=f"{additional_prompt_positive}, photorealistic, perfect body, beautiful skin, realistic skin, natural skin",
    negative_prompt=f"{additional_prompt_negative}, ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings",
    width=512, height=512
)

for idx, image in enumerate(images):
    image = image.resize((masked_image.width, masked_image.height))
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.axis('off')
    plt.savefig(os.path.join(dir, f'final_{idx}.jpg'), bbox_inches="tight", dpi=300, pad_inches=0.0)
